In [28]:
import requests
import bs4
from bs4 import BeautifulSoup
!pip install selenium
!pip install ChromeDriver
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

     |████████████████████████████████| 14.3MB 3.7MB/s 


In [0]:
URL = 'https://www.indeed.com/jobs?q=data%20scientist&l=Atlanta%2C%20GA&advn=5432824348636674'
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.content, 'html.parser')
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

In [0]:
def get_soup(url):
    driver = webdriver.Chrome()
    driver.get(url)
    counter = 0
    while True:
        try:
            click_me = driver.find_element_by_xpath('/html/body/main/section[1]/button')
            time.sleep(2)
            click_me.click()
            counter += 1
            if counter == 40:
                break
        except Exception as e:
            break
    my_var = driver.find_elements_by_class_name('result-card__full-card-link')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    return soup

In [0]:
def grab_job_links(soup):
    urls = []
    for link in soup.find_all('span', {'class':'company'}):
      partial_url = link.a.get('href')
      url = 'https://www.indeed.com' + partial_url
      urls.append(url)
      time.sleep(1)
    return urls

In [0]:
def get_urls(position, num_pages, location):
    """
    Get all the job posting URLs resulted from a specific search.
    
    Parameters:
        position: job title to query
        num_pages: number of pages needed
        location: city to search in
    
    Returns:
        urls: a list of job posting URL's (when num_pages valid)
        max_pages: maximum number of pages allowed ((when num_pages invalid))
    """
    # We always need the first page
#     base_url = f'https://www.indeed.com/jobs?q={job}&l={city}&start='
#     soup = get_soup(base_url)
#     urls = grab_job_links(soup)
    
#     driver = webdriver.Chrome()
#     driver.get(url)
    base_url="https://www.indeed.com/jobs?q="
    position=position.replace(" ", "%20").strip()
    location=location.replace(" ", "%20").strip()
    url = f'https://www.indeed.com/jobs?q={position}&l={location}%2C%20GA&advn=5432824348636674&vjk=9f2697c604f96ef1'
    soup = get_soup(url)
    
    
    
    # Get the total number of postings found 
    posting_count_string = soup.find(name='div', attrs={'id':"searchCount"}).get_text()
    posting_count_string = posting_count_string[posting_count_string.find('of')+2:].strip()
    #print('posting_count_string: {}'.format(posting_count_string))
    #print('type is: {}'.format(type(posting_count_string)))
    
    try:
        posting_count = int(posting_count_string)
    except ValueError: # deal with special case when parsed string is "360 jobs"
        posting_count = int(re.search('\d+', posting_count_string).group(0))
        #print('posting_count: {}'.format(posting_count))
        #print('\ntype: {}'.format(type(posting_count)))
    finally:
        posting_count = 330 # setting to 330 when unable to get the total
        pass
    
    # Limit nunmber of pages to get
    max_pages = round(posting_count / 10) - 3
    if num_pages > max_pages:
        print('returning max_pages!!')
        return max_pages
    
        # Additional work is needed when more than 1 page is requested
    if num_pages >= 2:
        # Start loop from page 2 since page 1 has been dealt with above
        for i in range(2, num_pages+1):
            num = (i-1) * 10
            base_url = f'https://www.indeed.com/jobs?q={job}&l={city}&start={num}'
            try:
                soup = get_soup(base_url)
                # We always combine the results back to the list
                urls += grab_job_links(soup)
            except:
                continue

    # Check to ensure the number of urls gotten is correct
    #assert len(urls) == num_pages * 10, "There are missing job links, check code!"

    return urls     

In [0]:
def get_posting(urls):    
    position_list = []
    company_list = []
    location_list = []
    description_list = []
    
    for url in urls:
      soup = get_soup(url)
      try:
        position = soup.find(name='div', attrs={'class':'vjs-jobtitle'})
        position_list.append(position.lower())
                
        company = soup.find(name='div', attrs={'class':'vjs-cn'}).get_text()
        company_list.append(company.lower())
                
        location = soup.find(name='div', attrs={'class':'vjs-loc'}).get_text()
        location_list.append(location.lower())
                
        decription = soup.find(name='div', attrs={'class':'vjs-content'}).get_text()
        description_list.append(description.lower())
      except AttributeError:
        position_list.pop()
        continue
                
      df = pd.DataFrame({'position': position_list,
                         'company': company_list,
                         'location': location_list,
                         'description': description})
    return df

In [29]:
locations = ['New+York','Chicago','San+Francisco'] #, 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Kansas+City', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder', 'Detroit', 'Minneapolis', 'Charlotte']
positions = ['data scientist', 'data engineer', 'data analyst', 'machine learning']

for location in locations:
  for position in positions:
    print(f'Starting {location} {position}')
    get_urls(position, 10, location)
    df = get_posting(urls)
    df.to_csv(f'df_{location}_{position}.csv', index = False)
    print(f'{location} {pos} complete')

Starting New+York data scientist


WebDriverException: ignored